In [1]:
import cvxpy as cp
import numpy as np

In [2]:
# Number of Transmitters and receivers
n = 5

# Maximimum power for each transmitter
P_max = np.repeat(3.0, n)

# Self noise (power) produced at each receiver
sigma = np.repeat(0.5, n)

# Total power received at each receiver
P_rc = np.repeat(5.0, n)

# np.repeat(x, 5) = np.array([x, x, x, x, x])

In [3]:
# Given Path gain matrix
G = np.array([[1.0, 0.1, 0.2, 0.1, 0.0],
              [0.1, 1.0, 0.1, 0.1, 0.0],
              [0.2, 0.1, 2.0, 0.2, 0.2],
              [0.1, 0.1, 0.2, 1.0, 0.1],
              [0.0, 0.0, 0.2, 0.1, 1.0]])

# Groups: {1, 2} and {3, 4, 5}
gp = np.array([[1,1,0,0,0],[0,0,1,1,1]])

# Power limit on groups
P_gp = np.array([4.0, 6.0])

# An accuracy of 0.05 in SINR
max_error = 0.05

In [4]:
# Power transmitted by each transmitter
p = cp.Variable(n)

# Signal power at each receiver
S = cp.multiply(np.diag(G), p)          # np.diag returns diagonal entries of a matrix

# Interference power at each receiver
I = (G @ p) - S

In [5]:
# min SINR = 1/alpha
alpha = cp.Parameter()

# maximizing (min SINR) = maximizing (1/alpha) = minimizing (alpha)
objective = cp.Minimize(alpha)

# first 4 constraints are given and fifth follows from bisection method
constraints = [p>=0, p<=P_max, (gp @ p) <= P_gp, (G @ p) <= P_rc, I + sigma <= alpha*S]

In [6]:
# Bisection Method

L = 0       # lower bound
U = 1000    # upper bound

while(U-L >=  max_error):
  alpha.value = (L+U)/2
  prob = cp.Problem(objective, constraints)
  prob.solve()

  # To check the feasibility
  if prob.status == 'optimal':
    U = alpha.value
    optimal_p = p.value
    max_SINR = 1/alpha.value
    
  else:
    L = alpha.value

In [7]:
print("Transmitter powers: ", optimal_p)
print("Maximum SINR: ", max_SINR)

Transmitter powers:  [2.05325577 1.85690341 1.5972308  2.3231306  1.85569467]
Maximum SINR:  1.6384
